In [ ]:
# pip install ib-insync # must do this first before trying to import it

In [1]:
import ib_insync
from ib_insync import *
import requests
import pandas as pd
#import schedule
import time
import datetime;
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
# https://stackoverflow.com/questions/44933518/how-to-remove-runtimewarning-errors-from-code

util.startLoop()  # only use in interactive environments (i.e. Jupyter Notebooks)
# https://algotrading101.com/learn/ib_insync-interactive-brokers-api-guide/
# https://www.reddit.com/r/interactivebrokers/comments/1cccg0t/ib_insync_reqhistoricaldata_with_keepuptodatetrue/
# https://github.com/erdewit/ib_insync/issues/653

In [3]:
#WORKING
counter_of_numbers = 0
ib = IB()
ib.connect(host='127.0.0.1', port=7496, clientId=0)
# https://www.elitetrader.com/et/threads/cant-connect-to-api-interactive-brokers.361205/
# needed to change socket port to 7496 and uncheck the Read-Only API box

<IB connected to 127.0.0.1:7496 clientId=0>

In [5]:
#12:07
positions = ib.positions()
positions

[Position(account='U12848716', contract=Stock(conId=700941285, symbol='NUVOQ', exchange='PINK', currency='USD', localSymbol='NUVOQ', tradingClass='NUVOQ'), position=275.0, avgCost=0.24124875),
 Position(account='U12848716', contract=Stock(conId=670777621, symbol='SVMH', exchange='NASDAQ', currency='USD', localSymbol='SVMH', tradingClass='NMS'), position=1.0, avgCost=0.0597)]

In [7]:
#12:08
tester = ib.trades()
tester
# these were cancelled before market open but had been written beforehand.. so..

[Trade(contract=Stock(conId=670777621, symbol='SVMH', right='?', exchange='SMART', currency='USD', localSymbol='SVMH', tradingClass='NMS'), order=Order(permId=241210818, action='SELL', totalQuantity=1.0, orderType='LMT', lmtPrice=0.07, auxPrice=0.0, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U12848716', clearingIntent='IB', cashQty=0.0, dontUseAutoPriceForHedge=True, autoCancelDate='20250630 17:00:00 EST', filledQuantity=0.0, refFuturesConId=2147483647, shareholder='Not an insider or substantial shareholder'), orderStatus=OrderStatus(orderId=0, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[], advancedError=''),
 Trade(contract=Stock(conId=586262891, symbol='SYRS', right='?', exchange='SMART', currency='USD', localSymbol='SYRS', tradingClass='NMS'), order=Order(per

In [19]:
# lets try to see if fillEvent works with MarketOrder and what fill information pops up
pending_stop_loss_orders = pd.DataFrame()

def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        print(fill)
        pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, fill], ignore_index=True)
                
contract = Stock("APDN", 'SMART', 'USD')
marketOrder = MarketOrder('BUY', 1)
marketOrderTrade = ib.placeOrder(contract, marketOrder)
trade_details = marketOrderTrade

# CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
new_row = pd.DataFrame({"Ticker": [trade_details.contract.symbol], "Quantity": [1], "SLTarget": [0.015]})
pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)



marketOrderTrade.fillEvent += order_status

# lets see what the storage looks like
print(pending_stop_loss_orders)

# 12:18 - nothing appeared to happen

  Ticker  Quantity  SLTarget
0   APDN         1     0.015


In [21]:
#12:19
positions = ib.positions()
positions

[Position(account='U12848716', contract=Stock(conId=700941285, symbol='NUVOQ', exchange='PINK', currency='USD', localSymbol='NUVOQ', tradingClass='NUVOQ'), position=275.0, avgCost=0.24124875),
 Position(account='U12848716', contract=Stock(conId=670777621, symbol='SVMH', exchange='NASDAQ', currency='USD', localSymbol='SVMH', tradingClass='NMS'), position=1.0, avgCost=0.0597),
 Position(account='U12848716', contract=Stock(conId=699197691, symbol='APDN', exchange='NASDAQ', currency='USD', localSymbol='APDN', tradingClass='SCM'), position=6.0, avgCost=0.20737583333333334)]

In [23]:
#12:19
filled = ib.fills()
filled

[Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0000ddc2.67812bcb.01.01', time=datetime.datetime(2025, 1, 10, 17, 13, 37, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.199, permId=1655614144, clientId=0, orderId=3, liquidation=0, cumQty=1.0, avgPrice=0.199, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), commissionReport=CommissionReport(execId='0000ddc2.67812bcb.01.01', commission=0.002036, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2025, 1, 10, 17, 13, 37, 297013, tzinfo=datetime.timezone.utc)),
 Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0001091d.67812ede.01.01', time=datetime.datetime(2025, 1, 10, 17, 14, 40, tzinfo=datetime.tim

In [25]:
#12:20
executions = ib.executions()
executions

[Execution(execId='0000ddc2.67812bcb.01.01', time=datetime.datetime(2025, 1, 10, 17, 13, 37, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.199, permId=1655614144, clientId=0, orderId=3, liquidation=0, cumQty=1.0, avgPrice=0.199, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2),
 Execution(execId='0001091d.67812ede.01.01', time=datetime.datetime(2025, 1, 10, 17, 14, 40, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.2, permId=1655614145, clientId=0, orderId=4, liquidation=0, cumQty=1.0, avgPrice=0.2, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2),
 Execution(execId='0000e494.67820280.01.01', time=datetime.datetime(2025, 1, 10, 17, 15, 7, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='DARK', side='BOT', shares=1.0, price=0.198, permId=1655614146, clientId=0, orderId=5, liquidation=0, cumQty=1.0, avgPri

In [27]:
#12:22
open_trades = ib.openTrades()
open_trades

[]

In [39]:
# now lets print something when order_status has been hit
pending_stop_loss_orders = pd.DataFrame()

def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        for index, row in pending_stop_loss_orders.iterrows():
            print(row)   
                
contract = Stock("CDT", 'SMART', 'USD')
limitOrder = LimitOrder('BUY', 1, 0.1228)
limitOrderTrade = ib.placeOrder(contract, limitOrder)
trade_details = limitOrderTrade

# CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
new_row = pd.DataFrame({"Ticker": [trade_details.contract.symbol], "Quantity": [1], "SLTarget": [0.015]})
pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)

# lets see what the storage looks like
#print(pending_stop_loss_orders)

limitOrderTrade.fillEvent += order_status
#12:29

In [41]:
#12:36
filled = ib.fills()
filled

[Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0000ddc2.67812bcb.01.01', time=datetime.datetime(2025, 1, 10, 17, 13, 37, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.199, permId=1655614144, clientId=0, orderId=3, liquidation=0, cumQty=1.0, avgPrice=0.199, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), commissionReport=CommissionReport(execId='0000ddc2.67812bcb.01.01', commission=0.002036, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2025, 1, 10, 17, 13, 37, 297013, tzinfo=datetime.timezone.utc)),
 Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0001091d.67812ede.01.01', time=datetime.datetime(2025, 1, 10, 17, 14, 40, tzinfo=datetime.tim

In [45]:
# now lets print something when order_status has been hit
def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        print(fill())
                
contract = Stock("CDT", 'SMART', 'USD')
limitOrder = LimitOrder('BUY', 1, 0.1218)
limitOrderTrade = ib.placeOrder(contract, limitOrder)
trade_details = limitOrderTrade




limitOrderTrade.fillEvent += order_status
#12:39 it corssed over and nothing appeared to happen wiht the print

In [47]:
#12:40
filled = ib.fills()
filled

[Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0000ddc2.67812bcb.01.01', time=datetime.datetime(2025, 1, 10, 17, 13, 37, tzinfo=datetime.timezone.utc), acctNumber='U12848716', exchange='IBKRATS', side='BOT', shares=1.0, price=0.199, permId=1655614144, clientId=0, orderId=3, liquidation=0, cumQty=1.0, avgPrice=0.199, orderRef='', evRule='', evMultiplier=0.0, modelCode='', lastLiquidity=2), commissionReport=CommissionReport(execId='0000ddc2.67812bcb.01.01', commission=0.002036, currency='USD', realizedPNL=0.0, yield_=0.0, yieldRedemptionDate=0), time=datetime.datetime(2025, 1, 10, 17, 13, 37, 297013, tzinfo=datetime.timezone.utc)),
 Fill(contract=Stock(conId=699197691, symbol='APDN', exchange='IBKRATS', currency='USD', localSymbol='APDN', tradingClass='SCM'), execution=Execution(execId='0001091d.67812ede.01.01', time=datetime.datetime(2025, 1, 10, 17, 14, 40, tzinfo=datetime.tim

In [ ]:
test_pd = pd.DataFrame()

In [ ]:


new_row = pd.DataFrame({"Ticker": ["AAAA"], "Quantity": [1], "SLTarget": [0.015]})
#test_pd = pd.concat([test_pd, new_row], ignore_index=True)

print(test_pd)

for index, row in test_pd.iterrows():
            if(index == 2):
                test_pd.drop(index, inplace=True) 
                # https://stackoverflow.com/questions/28876243/how-to-delete-the-current-row-in-pandas-dataframe-during-df-iterrows

print(test_pd)

In [ ]:
pending_stop_loss_orders = pd.DataFrame()

In [ ]:
def order_status(trade):
    if trade.orderStatus.status == 'Filled':
        fill = trade.fills[-1]
        for index, row in pending_stop_loss_orders.iterrows():
            if(row['Ticker'] == fill.contract.symbol and row['Quantity'] == fill.execution.shares):
                
                contract = Stock(fill.contract.symbol, 'SMART', 'USD')
                stopOrder = StopOrder('SELL', fill.execution.shares, row['SLTarget'])
                stopOrderTrade = ib.placeOrder(contract, stopOrder)
                
                pending_stop_loss_orders.drop(index, inplace=True)   # no longer useful to keep (does this work?)
                break
                
                
contract = Stock("SYRS", 'SMART', 'USD')
limitOrder = LimitOrder('BUY', 1, 0.19)
limitOrderTrade = ib.placeOrder(contract, limitOrder)
trade_details = limitOrderTrade

# CREATE A DATAFRAME THAT TRACKS STOP LIMIT ORDERS MADE ALONG WITH THEIR STOP LOSS TARGETS
new_row = pd.DataFrame({"Ticker": [trade_details.contract.symbol], "Quantity": [1], "SLTarget": [0.015]})
pending_stop_loss_orders = pd.concat([pending_stop_loss_orders, new_row], ignore_index=True)

# lets see what the storage looks like
print(pending_stop_loss_orders)

limitOrderTrade.fillEvent += order_status

In [ ]:
# lets see what executions gives in information and parse it up
tester = ib.executions()
tester

In [ ]:
# lets see what fills gives in information and parse it up
tester = ib.fills()
tester

In [ ]:
# lets see what openTrades gives in information and parse it up
tester = ib.openTrades()
tester